<a href="https://colab.research.google.com/github/nvaikunt/PromptBasedReranking/blob/main/BaselineTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nvaikunt/PromptBasedReranking.git

Cloning into 'PromptBasedReranking'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 51 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:28
🔁 Restarting kernel...


In [1]:
!conda --version

conda 4.14.0


In [2]:
%cd PromptBasedReranking/

/content/PromptBasedReranking


In [ ]:
!bash setup.sh

In [4]:
!python Baseline/download_dpr_data.py --resource data.wikipedia-split.psgs_w100

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [5]:
!python Baseline/download_dpr_data.py --resource data.retriever-outputs.dpr.nq-train

Requested resource from %s https://www.dropbox.com/s/6g4erof4ifg8xea/nq-train.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.json

Downloaded to %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/dpr


In [6]:

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import tokenizers
import datasets 
import pandas

wikipedia_txt = datasets.load_dataset("csv", data_files='downloads/data/wikipedia-split/psgs_w100.tsv', delimiter='\t')


Using custom data configuration default-cb3c6fef8f5e82e0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cb3c6fef8f5e82e0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
wikipedia_txt

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'title'],
        num_rows: 21015324
    })
})

In [8]:
nq_open = datasets.load_dataset("json",data_files="downloads/data/retriever-outputs/dpr/nq-train.json")

Using custom data configuration default-a7ea593b34a6dc21


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
nq_open["train"][2]["question"]

'which part of earth is covered with water'

In [10]:
def get_top_k_pos(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k = []
  for ctx in ctxs:
    if ctx["has_answer"]:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k.append((text, "true"))
    if len(top_k) == k:
      break
  if len(top_k) == 0: 
    return []
  while len(top_k) < k:
    top_k.extend(top_k[:(k - len(top_k))])

  return top_k[:k]
  
def get_top_k_pos_neg(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k_pos = []
  top_k_neg = []
  for ctx in ctxs:
    if ctx["has_answer"] and len(top_k_pos) < k:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_pos.append((text, "true"))
    if not ctx["has_answer"] and len(top_k_neg) < k: 
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_neg.append((text, "false"))
    if len(top_k_pos) == k and len(top_k_neg):
      break 
  if len(top_k_pos) == 0: 
    return []
  while len(top_k_pos) < k:
    top_k_pos.extend(top_k_pos[:(k - len(top_k_pos))])
  while len(top_k_neg) < k:
    top_k_neg.extend(top_k_neg[:(k - len(top_k_neg))])
 
  return top_k_pos + top_k_neg



In [11]:
def create_pos_txt_col(example, k, txt_database):
  return {"pos_text": get_top_k_pos(example, k, txt_database)}

def create_pos_neg_txt_col(example, k, txt_database):
  return {"pos_neg_text": get_top_k_pos_neg(example, k, txt_database)}


In [12]:
from functools import partial
nq_open["train"] = nq_open["train"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)

#2:   0%|          | 0/19792 [00:00<?, ?ex/s]

#3:   0%|          | 0/19792 [00:00<?, ?ex/s]

#1:   0%|          | 0/19792 [00:00<?, ?ex/s]

#0:   0%|          | 0/19792 [00:00<?, ?ex/s]

In [13]:
nq_open["train"] = nq_open["train"].map(partial(create_pos_neg_txt_col, k=10, txt_database=wikipedia_txt["train"]), num_proc=4)

#0:   0%|          | 0/19792 [00:00<?, ?ex/s]

#1:   0%|          | 0/19792 [00:00<?, ?ex/s]

#2:   0%|          | 0/19792 [00:00<?, ?ex/s]

#3:   0%|          | 0/19792 [00:00<?, ?ex/s]

In [ ]:
nq_open["train"][0]["pos_text"]

In [15]:
from tqdm import tqdm
def create_ranking_loss_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  with_answer = 0 
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Question: {question} Passage: {text[0]} Relevant: " for text in texts]
    current_targets = [text[1] for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1 
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}

def create_q_gen_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1     
  k = [len(targets)/(with_answer)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos": k}

def create_q_gen_ranking_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1   
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}



In [16]:
training_dict = create_q_gen_baseline_examples(nq_open["train"])

100%|██████████| 79168/79168 [15:16<00:00, 86.38it/s]


In [17]:
q_gen_dataset = datasets.Dataset.from_dict(training_dict)

In [18]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

model_checkpoint = "google/t5-base-lm-adapt"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [21]:
q_gen_dataset[2000]

{'inputs': 'Passage: The Way You Look Tonight "The Way You Look Tonight" is a song from the film "Swing Time" that was performed by Fred Astaire and written by Dorothy Fields and Jerome Kern. It won the Academy Award for Best Original Song in 1936. Fields remarked, "The first time Jerry played that melody for me I went out and started to cry. The release absolutely killed me. I couldn\'t stop, it was so beautiful." In the movie, Astaire sang "The Way You Look Tonight" to Ginger Rogers while she was washing her hair in an adjacent room. His recording reached the Please write a question based on this passage',
 'targets': 'who sings the song the way you look tonight',
 'k_pos': 20.0}

In [116]:
def preprocess_function(examples, max_input_length, max_target_length, input_col):
    model_inputs = tokenizer(
        examples[input_col],
        max_length=max_input_length,
        truncation=True,  padding="max_length"
    )
    labels = tokenizer(text_target=examples["targets"], max_length=max_target_length, truncation=True,  padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]
    return model_inputs

In [22]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [117]:
eval_subset = q_gen_dataset.select(range(50))
eval_dataset = eval_subset.map(partial(preprocess_function, max_input_length=300, max_target_length=50, input_col='inputs'), batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [142]:
#eval_dataset = eval_dataset.remove_columns(['inputs', 'targets', 'k_pos'])
eval_inputs = torch.tensor(eval_dataset["input_ids"])
eval_masks = torch.tensor(eval_dataset["attention_mask"])
target_eval_labels = torch.tensor(eval_dataset["labels"])
target_eval_label_mask= torch.tensor(eval_dataset["labels_mask"])

In [168]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
eval_masks.to(device)
eval_inputs.to(device)
target_eval_labels.to(device)
target_eval_label_mask.to(device)
outputs = model(input_ids=eval_inputs, labels=target_eval_labels, attention_mask=eval_masks, decoder_attention_mask=target_eval_label_mask)

In [214]:
import torch.nn 
logits = outputs.logits
log_softmax = torch.nn.LogSoftmax(dim=-1)
log_soft = log_softmax(logits)

In [ ]:
labels = target_eval_labels.unsqueeze(2)
log_soft = log_soft.gather(2, labels).squeeze(2)
log_soft = log_soft.mean(dim=1)
topk_scores, indexes = torch.topk(log_soft, k=len(log_soft))
topk_scores
eval_subset[indexes]


In [220]:
def evaluate_questions(validation, k, model, tokenizer, batch_size, dataset_generator=None):
  if not dataset_generator:
    dataset_generator = create_q_gen_baseline_examples
  for i in tqdm(range(0, len(validation), batch_size)):
    



torch.Size([50, 50, 32128])

In [229]:
dataset_gen = datasets.Dataset.from_dict(create_q_gen_baseline_examples(nq_open["train"].select(range(50))))


100%|██████████| 50/50 [00:00<00:00, 89.89it/s]


In [232]:
dataset_gen[49]

{'inputs': "Passage: in oceans and other large bodies of water, with 1.6% of water below ground in aquifers and 0.001% in the air as vapor, clouds, and precipitation. Oceans hold 97% of surface water, glaciers, and polar ice caps 2.4%, and other land surface water such as rivers, lakes, and ponds 0.6%. Additionally, a minute amount of the Earth's water is contained within biological bodies and manufactured products. An ocean is a major body of saline water, and a principal component of the hydrosphere. Approximately 71% of the Earth's surface (an area of some 361 million square kilometers) is covered by ocean, Please write a question based on this passage",
 'targets': 'which part of earth is covered with water',
 'k_pos': 20.0}